![Workshop Banner](assets/S1_M3.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CLDiego/SPE_GeoHackathon_2025/blob/main/S1_M3_GradioAgent.ipynb)

***
# Session 01 // Module 03: Agent Interfaces

This module takes the chat pipeline built in Module 02 and ships it as a usable browser app with Gradio. You’ll learn how to wrap your LangChain chat workflow into a small agent-like class, attach lightweight memory, and create a polished UI with events and session management.

***

# 1. What is an agent?

An agent is an LLM-driven decision-maker that plans steps and chooses tools to reach a goal. Instead of a fixed, linear chain, an agent decides what to do next at runtime (think → act → observe loop).

- Core pieces:
  - LLM (the “brain”)
  - Prompt/policy (instructions + scratchpad)
  - Tools (functions/APIs the model can call)
  - Memory (optional; keeps context)
  - Executor/loop (runs until done)

- When to use:
  - Multi-step tasks with branching/uncertainty
  - When you need external tools/data (RAG, web, code, math)
  - When the model should decide the next action

- Chain vs Agent:
  - Chain: predefined flow (prompt → model → parser)
  - Agent: dynamic, tool-using loop decided at runtime

In this notebook, we implement a memory-enabled chat workflow and expose it in a UI. This is the foundation you’ll later wrap with an Agent executor and tools.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Environment setup
!pip -q install langchain langchain-core langchain-community langchain-huggingface torch gradio
!pip -q install requests bitsandbytes transformers

In [ ]:
# Hugging Face API token
# Retrieving the token is required to get access to HF hub
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# 2. Designing a simple agent class

To simplify usage and integration, we design a simple `ChatAgent` class that wraps the LangChain components explored earlier.

Key features:
- Initialize with model, prompt, and memory.
- Encapsulate system prompts and chains.
- Offer simple methods (chat, clear_memory, get_history, create_new_session).
- Swap models without changing the interface.

> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/reminder.svg" width="20"/> Class design tips:
> - Keep system prompts centralized and editable.
> - Return clean strings to the UI layer.
> - Add basic try/except for robust demos.

By the end we will have a reusable `ChatAgent` class that can be easily extended with tools and wrapped in an Agent executor later. To use it, simply create an instance and call the `chat` method with user input:

```python
response = agent.chat("What role does well logging play?")
```

In [ ]:
from langchain_huggingface import ChatHuggingFace
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
import uuid

class ModernGeoscienceChatAgent:
    def __init__(self, chat_model: ChatHuggingFace):
        self.chat_model = chat_model
        self.store = {}
        
        # Enhanced system prompt
        self.system_prompt = """
You are Dr. GeoBot, a friendly and knowledgeable geoscience expert specializing in:
- Geophysics and seismic interpretation
- Petroleum geology and reservoir engineering  
- Well logging and formation evaluation
- Hydrocarbon exploration and production
- Geomechanics and drilling engineering

Guidelines:
- Provide accurate, helpful answers about geoscience topics
- Use technical terms but explain them when needed
- Be conversational and engaging
- Keep responses focused and informative
- If unsure, acknowledge limitations honestly
- Reference previous conversation when relevant
"""
        
        # Create prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}")
        ])
        
        # Create chain
        self.chain = self.prompt | self.chat_model | StrOutputParser()
        
        # Create conversational chain with memory
        self.conversational_chain = RunnableWithMessageHistory(
            self.chain,
            self.get_session_history,
            input_messages_key="question",
            history_messages_key="history",
        )
    
    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]
    
    def chat(self, question: str, session_id: str = "default") -> str:
        """Process a question and return a response"""
        try:
            config = {"configurable": {"session_id": session_id}}
            response = self.conversational_chain.invoke(
                {"question": question},
                config=config
            )
            return response.strip()
        except Exception as e:
            return f"I apologize, but I encountered an error: {str(e)}"
    
    def clear_memory(self, session_id: str = "default"):
        """Clear conversation history for a session"""
        if session_id in self.store:
            self.store[session_id].clear()
    
    def get_history(self, session_id: str = "default") -> list:
        """Get conversation history for a session"""
        if session_id in self.store:
            return self.store[session_id].messages
        return []
    
    def create_new_session(self) -> str:
        """Create a new conversation session"""
        return str(uuid.uuid4())


## 2.1 Model loading and decoding parameters

We use a Transformers text-generation pipeline and pass it through LangChain.

Common knobs:
- `max_new_tokens`: cap generated tokens (100–200 for chat)
- `temperature`: randomness (0.2–0.7 helpful; >0.9 creative)
- `top_p` or `top_k`: sampling filters; use one for easier tuning
- `repetition_penalty` or `no_repeat_ngram_size`: reduce loops


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
import torch

model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    quantization_config=quant_config
)

# Set pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    temperature=0.2,
    do_sample=True, # Sampling enables more diverse outputs
    pad_token_id=tokenizer.eos_token_id,
    return_full_text=False # The generated text will not include the prompt
)

# Create LangChain LLM
llm = HuggingFacePipeline(pipeline=pipe)
chat_model = ChatHuggingFace(llm=llm)

# Create the modern chat agent
chat_agent = ModernGeoscienceChatAgent(chat_model)
print("Modern GeoscienceChatAgent created successfully!")

In [ ]:
# Test the modern chat agent
print("=== Testing Modern GeoscienceChatAgent ===")

# Test conversation
questions = [
    "Hello! Can you explain what you specialize in?",
    "What is the difference between conventional and unconventional reservoirs?",
    "How do geophysicists use seismic data to find oil?",
    "What role does well logging play in this process?"
]

session_id = chat_agent.create_new_session()
print(f"Created session: {session_id[:8]}...\n")

for i, question in enumerate(questions, 1):
    print(f"{i}. Human: {question}")
    response = chat_agent.chat(question, session_id)
    print(f"   Dr. GeoBot: {response}")
    print("-" * 100)

# 3. Building a Gradio UI

Gradio is a popular open-source library to build web UIs for ML models with minimal code. The core idea is to define components (text boxes, buttons, chat windows) and wire them to Python functions via events. You can access the app in a browser and share it via a public link.

Here is a link to the [Gradio documentation](https://gradio.app/docs/) for more details.

Core components:
- `gr.Blocks`: Page/layout container.
- `gr.Chatbot`: Conversation pane (list of (user, bot) tuples).
- `gr.Textbox`: Input field for user messages.
- `gr.Button`: Send, Clear, Load examples.
- Events: `.submit` and `.click` wire UI to Python callbacks.

UX tips:
- Keep answers concise; use memory to reduce repetition.
- Provide example prompts to guide first-time users.
- Add a “Clear chat” to reset session/memory.

***

> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> Snippet: Minimal Gradio event handlers

```python
def respond(message, history):
    reply = agent.chat(message, session_id)
    history.append((message, reply))
    return "", history

msg.submit(respond, [msg, chatbot], [msg, chatbot])
send_btn.click(respond, [msg, chatbot], [msg, chatbot])
clear_btn.click(lambda: agent.clear_memory(session_id) or [], outputs=chatbot)
```

In [ ]:
from typing import List, Tuple

# Create a new chat agent for the interface
gradio_agent = ModernGeoscienceChatAgent(chat_model)

# Global session management
current_session = gradio_agent.create_new_session()

def respond(message: str, history: List[Tuple[str, str]]) -> Tuple[str, List[Tuple[str, str]]]:
    """
    Process user message and return bot response
    """
    global current_session
    
    if not message.strip():
        return "", history
    
    # Get response from agent
    bot_response = gradio_agent.chat(message, current_session)
    
    # Add to chat history
    history.append((message, bot_response))
    
    return "", history

def clear_conversation() -> List[Tuple[str, str]]:
    """
    Clear conversation history and start new session
    """
    global current_session
    gradio_agent.clear_memory(current_session)
    current_session = gradio_agent.create_new_session()
    return []

def load_example(example: str) -> str:
    """
    Load example question into the textbox
    """
    return example


In [ ]:
import gradio as gr

# Create modern Gradio interface
with gr.Blocks(
    title="Dr. GeoBot - Advanced Geoscience Chat Assistant",
    theme=gr.themes.Monochrome(
        font=[gr.themes.GoogleFont("JetBrains Mono"), "monospace"],
    )
) as demo:
    
    gr.Markdown("""
    # 🌍 Dr. GeoBot - Your Advanced Geoscience Expert
    
    I'm an AI geoscience expert powered by modern LangChain. Ask me about:
    
    | **Geophysics** | **Petroleum Engineering** | **Well Logging** |
    |---|---|---|
    | Seismic interpretation | Reservoir characterization | Formation evaluation |
    | Gravity & magnetics | Hydrocarbon systems | Petrophysics |
    | Electromagnetics | Production optimization | Log analysis |
    
    💡 *I remember our conversation, so feel free to ask follow-up questions!*
    """)
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                value=[],
                height=500,
                show_label=False,
                bubble_full_width=False
            )
            
            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Ask me about geoscience topics...",
                    show_label=False,
                    scale=4,
                    container=False
                )
                send_btn = gr.Button("Send 📤", scale=1, variant="primary")
            
            with gr.Row():
                clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary")
                
        with gr.Column(scale=1):
            gr.Markdown("### 💡 Example Questions")
            
            example_questions = [
                "What is seismic inversion?",
                "Explain porosity vs permeability",
                "How do P-waves and S-waves differ?",
                "What is reservoir characterization?",
                "How does well logging work?",
                "What are the challenges in unconventional reservoirs?"
            ]
            
            for question in example_questions:
                example_btn = gr.Button(
                    question,
                    variant="secondary",
                    size="sm"
                )
                example_btn.click(
                    load_example,
                    inputs=[gr.State(question)],
                    outputs=msg
                )
    
    # Event handlers
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send_btn.click(respond, [msg, chatbot], [msg, chatbot])
    clear_btn.click(clear_conversation, outputs=chatbot)


In [ ]:
# Launch the interface
print("Launching modern Gradio interface...")
demo.launch(share=True, show_error=True)

# 4. Troubleshooting and performance

- Slow replies
  - Lower max_new_tokens; use smaller model; close other notebooks
- Repetition/rambling
  - Lower temperature; add repetition_penalty=1.1; cap turn count
- Tokenizer/pad errors
  - tokenizer.pad_token = tokenizer.eos_token
- BitsAndBytes import errors (Mac)
  - Skip 4-bit; load small model with torch_dtype=torch.float16 on MPS
- Memory not “sticking”
  - Ensure you pass config={"configurable": {"session_id": ...}} on each call